In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
year = 2022
refund = 112000.00
dividend = 0.00
expense  = 66_000.00

today = date.today()
today

datetime.date(2022, 2, 4)

### Restart and Run All Cells

### Record selection for transactions

In [2]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
ORDER BY sells.date DESC, name


In [3]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,MCS,2022,2,2021,3,14.70,16.70,-2.00,"5,000","73,500.00","83,500.00","-10,000.00",-11.98,"-10,347.75",HD
1,CKP,2022,2,2021,11,5.50,5.00,0.50,"10,000","55,000.00","50,000.00","5,000.00",10.00,"4,767.43",DTD
2,IP,2022,2,2021,12,22.00,20.20,1.80,"3,000","66,000.00","60,600.00","5,400.00",8.91,"5,119.60",DTD
3,NER,2022,1,2021,9,7.30,7.45,-0.15,"9,000","65,700.00","67,050.00","-1,350.00",-2.01,"-1,644.03",DTD
4,TOP,2022,1,2021,11,53.25,50.50,2.75,"2,000","106,500.00","101,000.00","5,500.00",5.45,"5,040.41",DTD
5,IMH,2022,1,2022,1,17.70,17.60,0.10,"6,000","106,200.00","105,600.00",600.00,0.57,130.89,DTD
6,MCS,2022,1,2019,11,14.60,16.70,-2.10,"5,000","73,000.00","83,500.00","-10,500.00",-12.57,"-10,846.64",HD
7,IVL,2022,1,2021,7,48.25,42.00,6.25,"3,000","144,750.00","126,000.00","18,750.00",14.88,"18,150.32",DTD
8,IVL,2022,1,2021,6,46.25,42.00,4.25,"3,000","138,750.00","126,000.00","12,750.00",10.12,"12,163.61",DTD
9,KBANK,2022,1,2021,12,140.00,136.88,3.12,800,"112,000.00","109,504.00","2,496.00",2.28,"2,005.39",DTD


In [4]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

sell_year  sell_month  buy_year  buy_month  name 
2022       1           2019      11         MCS     -10,846.64
                       2021      3          IVL       6,925.23
                                 6          IVL      24,327.22
                                 7          IVL      18,150.32
                                            SAT       7,998.06
                                 9          DCC       3,477.28
                                            EPG       2,095.90
                                            NER      -1,644.03
                                 10         SAT      10,009.39
                                 11         KBANK    11,194.84
                                            RCL       7,187.70
                                            SIS       3,098.42
                                            TOP       5,040.41
                                 12         IMH      26,246.40
                                            KBANK     2,005.39
     

In [5]:
sells_df.groupby(['name'])[['gross','net']].sum().head().sort_values(by=['gross'],ascending=[False])

,gross,net
name,,
IMH,"27,100.00","26,377.29"
IP,"5,400.00","5,119.60"
CKP,"5,000.00","4,767.43"
DCC,"4,000.00","3,477.28"
EPG,"2,200.00","2,095.90"


In [6]:
total_df = sells_df.groupby(['name'])[['gross','net']].sum()
total_df.nlargest(5, 'gross')[['gross','net']].style.format(format_dict)

,gross,net
name,,
IVL,"51,750.00","49,402.77"
IMH,"27,100.00","26,377.29"
SAT,"18,750.00","18,007.45"
KBANK,"14,688.00","13,200.23"
RCL,"7,500.00","7,187.70"


In [7]:
file_name = 'ytd-profit.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name

sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(csv_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(data_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(box_file)

In [8]:
sells_df.groupby(['sell_month']).net.sum()

sell_month
1   117,432.48
2      -460.72
Name: net, dtype: float64

In [9]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(116971.76000000001, 0.0, 116971.76000000001)

In [10]:
profit = sells_df.net.sum()
net = profit + dividend - expense
profit,dividend,expense, net

(116971.76000000001, 0.0, 66000.0, 50971.76000000001)

### Bottom line of up to date transactions

In [11]:
total = refund + net
total

162971.76